# Exercise 3

In this exercise we'll implement the full FEM-method on a given grid in two-dimensions with linear finite elements and the laplace operator. Let $\Omega$ be given and 
$$
\partial \Omega = \underset{ i \in I } { \overset{\circ}{ \bigcup }} \Gamma_i
$$
then as usual we define the BVP as:
$$
\left\{ \begin{array}{rll}
- \Delta u & = f \qquad & \text{for } x \in \Omega\\
u &= g_i \qquad & \text{for } x \in \Gamma_i
\end{array}\right.
$$
In contrast to the previous homework this exercise will be structured a bit differently. While (a) will be the main part, where you'll be asked to implement all necessary functions, (b) will include different test that will test your functions.

While your code should run with arbitrary $\Omega$, for testing we'll make the following choices:
$$
\left. \begin{array}{c}
\Omega = \Omega_0 \setminus \Omega_1 \quad \text{with } \Omega_0 := [0,3]^2 \; \text{and} \; \Omega_1 := [1,2]^2\\
\Gamma_0 = \partial \Omega_0 \quad \text{and} \quad \Gamma_1 = \partial \Omega_1\\
 g_0(x,y) = | x- \frac{3}{2}| \quad \text{and} \quad g_1(x,y) = | y - \frac{3}{2} |.
\end{array} \right.
$$
For convenience we will also choose $f \equiv 0$. However, as above your code should also work with $f \not \equiv 0 $.

As an example the coarse mesh should render like this:
![title](images/example_grid.png)



### a) 

First we need to implement the basic functions:

 - ```load_mesh```: Returns a "Mesh" object from a given path, where the grid is stored as ".csv" files.
 - ```gen_trafo2d```: Returns the determinant and the inverse of the ( constant ) jacobian $F_l$ of the transformation $T_l$. You can find the definition of this matrix in equation III.18 in the script. Takes a mesh ( of class Mesh ) and three integers $e_k = (i_1,i_2,i_3)$, which are indices to the nodes of mesh that forms the triangle as an input.
 - ```get_M_loc```: Returns the mass matrix with respect to local indices as $\mathbb{R}^{3 \times 3 }$. Takes the determinant and inverse of $F_l$ as an input.
 - ```get_S_loc```: Returns the stiffness matrix with respect to local indices. Same input as above.
 - ```get_global_matrix``` : Returns the global stiffness or mass matrix. Takes the mesh and a function that returns a matrix with respect to local indices as an input.
 - ```get_bv```: Returns a tuple of a Boolean vector which is true if the corresponding degree of freedom isn't determined by the boundary conditions and false elsewhere. And a vector, that stores the value for the degree of freedom if it determined by the boundary condition. Takes a mesh and list of callable functions $g_i$ ( see in the definition ).

<b>Hint:</b> ```numpy.genfromtxt``` and ```Mesh._make``` can be useful.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from collections import namedtuple
from scipy.sparse import dok_matrix
from scipy.sparse.linalg import spsolve 
from scipy.integrate import dblquad
%matplotlib qt

fields = { "elements" : int, "faces" : int , "nodes" : float}
Mesh = namedtuple( "Mesh", fields )

def load_mesh( path ):
    return Mesh._make( [ 
            np.genfromtxt( f"{path}/{field}.csv", delimiter=",", dtype=dtype ) 
        for field,dtype in fields.items()
    ] )

def get_M_loc( Fdet, Finv ):
    phi = ['(1-y1-y2)','y1','y2']
    M_l = np.zeros([3,3])   # Initialize the mass matrix M_l
    for i in range(3):
        for j in range(3):
            loc = locals()  # need to define a local variable dictionary and then we can use exec
            exec('f = lambda y1,y2:{}*{}'.format(phi[i],phi[j]))  # product pf phi_ki and phi_kj 
            f = loc['f']
            M_l[i,j] = abs(Fdet) * dblquad( f,0,1,0,lambda y1:1-y1)[0]  # integral of phi_ki * phi_kj over omega
    return M_l

def gen_trafo2d( mesh, e_k ):
    nodes  = mesh[2]    # get nodes
    p = nodes[e_k]      # get 3 points  
    F_l = np.array( [ [ p[1,0]-p[0,0], p[2,0]-p[0,0] ],[ p[1,1]-p[0,1], p[2,1]-p[0,1] ] ] )  # F_l (III.8)
    det_F  = F_l[0,0]*F_l[1,1] - F_l[0,1]*F_l[1,0]  # determinant of Jacobian matrix F_l  
    inv_F = np.array( [ [ p[2,1]-p[0,1], p[0,0]-p[2,0] ],[ p[0,1]-p[1,1], p[1,0]-p[0,0] ] ] ) / det_F # transpose inverse of F_l
    return det_F, inv_F

def get_bv( mesh , g_list=0 ):
    boolean_list = np.array([True for i in mesh[2]])
    boundary = mesh[1]
    for j in range(len(boundary)):
        boolean_list[boundary[j,0]]=False
        boolean_list[boundary[j,1]]=False  
    return tuple(boolean_list)

def get_S_loc( Fdet, Finv ):
    grad_phi_k1 = np.array([-1,-1])  # gradient of phi_k1 
    grad_phi_k2 = np.array([1,0])    # gradient of phi_k2
    grad_phi_k3 = np.array([0,1])    # gradient of phi_k3
    grad_phi = np.array([grad_phi_k1,grad_phi_k2,grad_phi_k3])
    A_l = np.zeros([3,3])  # Initialize the matrix A_l
    for i in range(3):
        for j in range(3):
            A_l[i,j] = np.dot(np.dot(grad_phi[i].T,np.dot(Finv,Finv.T)),grad_phi[j]) * abs(Fdet)/2 # analytic integral
    return A_l
            
def get_global_matrix( mesh , get_local ):
    nodes = mesh[2]     # nodes
    elements = mesh[0]  # elements
    dim = len(nodes)    # dimension of global matrix
    bv_tuple = get_bv( mesh , g_list=0 )  # boundary boolean tuple

    A = dok_matrix((dim,dim), dtype=np.float32)   # Initialize the global matarix A
    for i in range(dim):
        if bv_tuple[i] == False:
            A[i,i] = 1
        else: 
            index1,index2 = np.where(elements==i)
            for j in range( len(index1) ):   
                
                ele_l = elements[ index1[j] ]           # element_l (containing node i)
                detF, invF = gen_trafo2d( mesh, ele_l ) # get determinant and inverse of F_l
                local_matrix = get_local( detF, invF )  # get 3*3 local matrix 
                loc_ind_i = index2[j]                   # local index of node i in element l
                
                for k in range(3):
                    if bv_tuple[ ele_l[k] ] == True:
                        A[i,ele_l[k]] += local_matrix[loc_ind_i,k]        
                        
    return A


### b)


Test your code by creating:

 - ```plot_mesh```: Draw all triangles of the loaded mesh and plot the boundary colorcoded with $i$.
 - ```integrate_omega```: Compute $\int_\Omega 1 dx$ by using the determinant of the jacobian matrix of the transformation and the known integral over the reference element. Compare it to your analytic solution.
 - ```get_uh```: Computes the solution with the given grid, callable functions $f$ and $g_i$ ( as a list ). Return all coefficents of the solution not only the inner ones.
 
Finally use ```plot_trisurf``` to show your result.


In [2]:
def plot_mesh( mesh ):
    x = mesh[2].T[0] # x coordinate
    y = mesh[2].T[1] # y coordinate
    ele = mesh[0]    # elements
    triang = Triangulation(x,y,triangles=ele)
    z = np.zeros(len(x))
    fig, axs = plt.subplots()
    axs.tricontourf(triang, z)
    axs.triplot(triang, 'go-', color ='black')
    fig.tight_layout()
    edge = mesh[1]
    nodes = mesh[2]
    for i in range( len(edge) ):
        node1 = nodes[edge[i,0]] 
        node2 = nodes[edge[i,1]]
        x = [node1[0],node2[0]]
        y = [node1[1],node2[1]]
        if edge[i,2] == 1:
            plt.plot(x,y,c='blue',label='gamma_1',linewidth=3)
        else:
            plt.plot(x,y,c='yellow',label='gamma_0',linewidth=6)
    
def integrate_omega( mesh ):
    ref_int = 1/2  # knowing integral of reference omega
    ele = mesh[0]  # elements
    S = 0 
    for i in range(len(ele)):
        detF,invF = gen_trafo2d( mesh, ele[i] )
        S += abs(detF) * ref_int  # Sum of the integral of every element  
    return S  

def rhs_f(x1,x2):
    return 2 * x2 - x1  # right hand side function

def get_uh( mesh, f ):
    nodes = mesh[2]  # nodes
    A = get_global_matrix( mesh , get_S_loc )  # get A matrix
    M = get_global_matrix( mesh , get_M_loc )  # get M matrix
    bv = get_bv( mesh , g_list=0 ) # get boundary boolean tuple
    f_nodes=np.zeros( len(nodes) )
    for i in range( len(nodes) ):
        if bv[i] == True:
            f_nodes[i] =  f(nodes[i,0],nodes[i,1]) 
    f_n = M * np.array(f_nodes)  # get right hand side f_n
    alpha = spsolve( A,f_n )  # solve A*alpha=f_n
    return alpha

for path, volume_omega in [("mesh/coarse",8.), ("mesh/fine", 8.),]:
    mesh = load_mesh( path )
    plot_mesh(mesh)
    print( f"By trafo2d: { integrate_omega(mesh) }, analytic: { volume_omega }") 

mesh = load_mesh( "mesh/fine" )
u_h = get_uh( mesh, rhs_f )

# Plot here
nodes = mesh[2]
fig = plt.figure(figsize =(14, 9))
ax = plt.axes(projection ='3d')

ax.plot_trisurf(nodes.T[0],nodes.T[1],u_h)

By trafo2d: 7.999999999999998, analytic: 8.0
By trafo2d: 8.0, analytic: 8.0


C:\Users\qqsup\AppData\Roaming\Python\Python38\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
